In [11]:
# -*- coding: utf-8 -*-
"""
Mundo do Wumpus com Agente de Lógica Proposicional (Solucionável com Ouro)

Este script implementa o jogo Mundo do Wumpus usando Pygame.
O agente (caçador) utiliza uma base de conhecimento e regras de
lógica proposicional para inferir quais células do mapa são seguras,
perigosas ou desconhecidas.

Novas funcionalidades:
- Ouro (G) foi adicionado ao mapa como objetivo.
- O agente percebe um "Brilho" na casa do ouro.
- O objetivo é pegar o ouro e voltar à casa inicial para vencer.
- O problema é sempre solucionável: Wumpus, buracos e ouro não podem
  aparecer na casa inicial ou em suas adjacentes.
- O mapa sempre contém 1 Wumpus, 2 Buracos e 1 Ouro.
"""

import pygame
import sys
import random

# --- Configurações do Jogo ---
TILE_SIZE = 120
GRID_SIZE = 4
WIDTH, HEIGHT = TILE_SIZE * GRID_SIZE, TILE_SIZE * GRID_SIZE + 100
FPS = 30

# --- Cores ---
GRAY = (180, 180, 180)    # Célula Desconhecida
GREEN = (0, 200, 0)      # Célula Segura
RED = (200, 0, 0)        # Célula Perigosa
BLACK = (0, 0, 0)
WHITE = (255, 255, 255)  # Célula Já Visitada
BROWN = (139, 69, 19)
GOLD = (255, 215, 0)     # Cor do Ouro
BLUE = (0, 0, 200)

# --- Inicialização do Pygame ---
pygame.init()
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Mundo do Wumpus - Lógica Proposicional")
font = pygame.font.SysFont("Arial", 24)
status_font = pygame.font.SysFont("Arial", 28, bold=True)
icon_font = pygame.font.SysFont("Arial", 48)
danger_font = pygame.font.SysFont("Arial", 80, bold=True)

# --- Funções Auxiliares ---
def get_adjacent(x, y):
    """Retorna uma lista de células adjacentes válidas."""
    adj = []
    if x > 0: adj.append((x - 1, y))
    if x < GRID_SIZE - 1: adj.append((x + 1, y))
    if y > 0: adj.append((x, y - 1))
    if y < GRID_SIZE - 1: adj.append((x, y + 1))
    return adj

# --- Classe Principal do Mundo ---
class WumpusWorld:
    def __init__(self):
        self.reset()

    def reset(self):
        """Inicializa ou reinicia o estado do mundo e do agente."""
        self.start_pos = (0, GRID_SIZE - 1)
        self.agent_pos = self.start_pos
        self.has_gold = False
        self.game_over = False
        self.victory = False

        # Garante que o jogo seja solucionável, proibindo perigos e ouro no início
        forbidden_cells = {self.start_pos} | set(get_adjacent(self.start_pos[0], self.start_pos[1]))
        
        possible_cells = [(x, y) for x in range(GRID_SIZE) for y in range(GRID_SIZE)
                          if (x, y) not in forbidden_cells]
        
        # Define 1 Wumpus, 2 Buracos e 1 Ouro
        num_holes = 2
        self.wumpus = random.choice(possible_cells)
        
        # Garante que o ouro e o wumpus não ocupem a mesma célula para simplificar
        possible_cells_no_wumpus = list(possible_cells)
        possible_cells_no_wumpus.remove(self.wumpus)
        
        self.holes = random.sample(possible_cells_no_wumpus, k=num_holes)
        
        possible_cells_for_gold = list(possible_cells_no_wumpus)
        for h in self.holes:
            possible_cells_for_gold.remove(h)
        self.gold = random.choice(possible_cells_for_gold)

        self._world_percepts = self._generate_all_percepts()

        # --- Conhecimento do agente ---
        self.visited = set()
        self.safe = {self.start_pos}
        self.danger = set()
        self.unknown = set([(x, y) for x in range(GRID_SIZE) for y in range(GRID_SIZE)])
        self.unknown.remove(self.start_pos)
        self.knowledge_base = {}
        self.history = []

    def _generate_all_percepts(self):
        """Cria o mapa de percepções (informação do ambiente)."""
        percepts = {}
        for x in range(GRID_SIZE):
            for y in range(GRID_SIZE):
                fedor = any(neighbor == self.wumpus for neighbor in get_adjacent(x, y))
                vento = any(neighbor in self.holes for neighbor in get_adjacent(x, y))
                brilho = (x, y) == self.gold
                percepts[(x, y)] = (fedor, vento, brilho)
        return percepts

    def sense_at_current_pos(self):
        """O agente sente o ambiente na sua posição atual."""
        return self._world_percepts[self.agent_pos]

    def logical_update(self):
        """Atualiza a base de conhecimento e realiza inferências lógicas."""
        pos = self.agent_pos
        if pos in self.visited:
            return

        self.visited.add(pos)
        self.unknown.discard(pos)
        self.safe.add(pos)
        
        fedor, vento, brilho = self.sense_at_current_pos()
        self.knowledge_base[pos] = (fedor, vento, brilho)

        if brilho:
            self.has_gold = True
            print("OURO ENCONTRADO E COLETADO!")

        # Regra 1: Se não há percepções, todos os vizinhos são seguros.
        if not fedor and not vento:
            for neighbor in get_adjacent(pos[0], pos[1]):
                if neighbor not in self.visited:
                    self.safe.add(neighbor)
                    self.unknown.discard(neighbor)

        # Regra 2: Inferir perigo por eliminação.
        for cell, (f, v, b) in self.knowledge_base.items():
            adjacents = get_adjacent(cell[0], cell[1])
            if v: # Vento -> Poço
                unknown_neighbors = [n for n in adjacents if n in self.unknown]
                if len(unknown_neighbors) == 1:
                    self.danger.add(unknown_neighbors[0])
                    self.unknown.discard(unknown_neighbors[0])
            if f: # Fedor -> Wumpus
                unknown_neighbors = [n for n in adjacents if n in self.unknown]
                if len(unknown_neighbors) == 1:
                    self.danger.add(unknown_neighbors[0])
                    self.unknown.discard(unknown_neighbors[0])
                    
    def step(self):
        """Executa uma jogada do agente."""
        if self.game_over or self.victory:
            return
            
        self.logical_update()
        
        safe_unvisited = {c for c in self.safe if c not in self.visited}
        
        # Se pegou o ouro, o objetivo é voltar para o início pelo caminho seguro
        if self.has_gold:
            # Tenta encontrar o caminho de volta
            # (Simplificado: move-se para qualquer célula segura visitada)
            choices = {c for c in self.safe if c in get_adjacent(self.agent_pos[0], self.agent_pos[1])}
        else:
            # Se não tem o ouro, prioriza explorar células seguras e não visitadas
            choices = {c for c in safe_unvisited if c in get_adjacent(self.agent_pos[0], self.agent_pos[1])}
            if not choices:
                choices = safe_unvisited

        if choices:
            self.history.append(self.agent_pos)
            self.agent_pos = random.choice(list(choices))
            
            if self.agent_pos in self.holes or self.agent_pos == self.wumpus:
                print("GAME OVER! Agente morreu.")
                self.game_over = True
            elif self.has_gold and self.agent_pos == self.start_pos:
                print("VITÓRIA! O agente escapou com o ouro.")
                self.victory = True

    def back(self):
        """Volta para a posição anterior."""
        if self.history and not self.game_over and not self.victory:
            self.agent_pos = self.history.pop()

# --- Funções de Desenho ---
def draw_world(world):
    """Desenha o estado atual do mundo."""
    screen.fill(BROWN)
    
    for x in range(GRID_SIZE):
        for y in range(GRID_SIZE):
            rect = pygame.Rect(x * TILE_SIZE, y * TILE_SIZE, TILE_SIZE, TILE_SIZE)
            cell = (x, y)
            color = GRAY # Padrão: Desconhecido

            if cell in world.visited: color = WHITE
            elif cell in world.safe: color = GREEN
            elif cell in world.danger: color = RED
            
            pygame.draw.rect(screen, color, rect)

            # MOSTRA A LOCALIZAÇÃO REAL DOS ITENS (APENAS PARA O USUÁRIO)
            if cell == world.gold:
                g_text = danger_font.render("G", True, GOLD)
                text_rect = g_text.get_rect(center=rect.center)
                screen.blit(g_text, text_rect)
            
            if cell == world.wumpus:
                w_text = danger_font.render("W", True, (150, 0, 0))
                text_rect = w_text.get_rect(center=rect.center)
                screen.blit(w_text, text_rect)
            elif cell in world.holes:
                b_text = danger_font.render("B", True, BLACK)
                text_rect = b_text.get_rect(center=rect.center)
                screen.blit(b_text, text_rect)

            pygame.draw.rect(screen, BLACK, rect, 2)

            # Desenha as percepções das células visitadas
            if cell in world.knowledge_base:
                fedor, vento, brilho = world.knowledge_base[cell]
                if fedor: screen.blit(icon_font.render("F", True, BLACK), (rect.left + 5, rect.top))
                if vento: screen.blit(icon_font.render("V", True, BLUE), (rect.right - 35, rect.top))
                if brilho: pygame.draw.polygon(screen, GOLD, [(rect.centerx, rect.top + 5), (rect.right-5, rect.centery), (rect.centerx, rect.bottom-5), (rect.left+5, rect.centery)])

    # Desenha o agente
    ax, ay = world.agent_pos
    agent_center = (ax * TILE_SIZE + TILE_SIZE // 2, ay * TILE_SIZE + TILE_SIZE // 2)
    pygame.draw.circle(screen, BLUE, agent_center, TILE_SIZE // 3)
    
    draw_ui(world)

def draw_ui(world):
    """Desenha a interface de botões e status."""
    ui_area = pygame.Rect(0, HEIGHT - 100, WIDTH, 100)
    pygame.draw.rect(screen, BLACK, ui_area)
    
    # Botões
    back_btn = pygame.Rect(10, HEIGHT - 85, 160, 50)
    next_btn = pygame.Rect(WIDTH - 170, HEIGHT - 85, 160, 50)
    reset_btn = pygame.Rect(WIDTH // 2 - 80, HEIGHT - 90, 160, 60)

    pygame.draw.rect(screen, GRAY, back_btn, border_radius=10)
    pygame.draw.rect(screen, GRAY, next_btn, border_radius=10)
    pygame.draw.rect(screen, (200, 50, 50), reset_btn, border_radius=10)

    screen.blit(font.render("← Voltar", True, BLACK), (back_btn.centerx - 40, back_btn.centery - 12))
    screen.blit(font.render("Próximo →", True, BLACK), (next_btn.centerx - 45, next_btn.centery - 12))
    screen.blit(font.render("Reiniciar", True, WHITE), (reset_btn.centerx - 45, reset_btn.centery - 12))
    
    # Texto de status
    status_text = "Caçando o ouro..."
    status_color = WHITE
    if world.has_gold:
        status_text = "Ouro encontrado! Volte para (0,3)"
    if world.victory:
        status_text = "VOCÊ VENCEU!"
        status_color = GREEN
    elif world.game_over:
        status_text = "GAME OVER!"
        status_color = RED

    status_surf = status_font.render(status_text, True, status_color)
    screen.blit(status_surf, (ui_area.centerx - status_surf.get_width()//2, ui_area.top + 5))
    
    return back_btn, next_btn, reset_btn

# --- Loop Principal ---
def main():
    clock = pygame.time.Clock()
    world = WumpusWorld()

    running = True
    while running:
        back_btn, next_btn, reset_btn = draw_ui(world)

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
            if event.type == pygame.MOUSEBUTTONDOWN:
                if next_btn.collidepoint(event.pos): world.step()
                elif back_btn.collidepoint(event.pos): world.back()
                elif reset_btn.collidepoint(event.pos): world.reset()
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_RIGHT: world.step()
                if event.key == pygame.K_LEFT: world.back()
                if event.key == pygame.K_r: world.reset()

        draw_world(world)
        pygame.display.flip()
        clock.tick(FPS)

    pygame.quit()
    sys.exit()

if __name__ == "__main__":
    main()

OURO ENCONTRADO E COLETADO!
VITÓRIA! O agente escapou com o ouro.
OURO ENCONTRADO E COLETADO!
VITÓRIA! O agente escapou com o ouro.
OURO ENCONTRADO E COLETADO!
VITÓRIA! O agente escapou com o ouro.
OURO ENCONTRADO E COLETADO!
VITÓRIA! O agente escapou com o ouro.


SystemExit: 